In [1]:
import datetime
import pytz
import pandas as pd
import yfinance as yf
import numpy as np

In [4]:
frame_M1 = '1m'
frame_M15 = '15m'
frame_M30 = '30m'
frame_H1 = '1h'
frame_H4 = '4h'
frame_D1 = '1d'
frame_W1 = '1wk'

now = datetime.datetime.now()

# Matching your MT5 asset list with Yahoo tickers
assets = {
    'EURUSD': 'EURUSD=X',
    'USDCHF': 'USDCHF=X',
    'GBPUSD': 'GBPUSD=X',
    'USDCAD': 'USDCAD=X',
    'BTCUSD': 'BTC-USD',
    'ETHUSD': 'ETH-USD',
    'XAUUSD': 'XAUUSD=X',
    'XAGUSD': 'XAGUSD=X',
    'SP500m': '^GSPC',
    'UK100': '^FTSE'
}

In [14]:
# def get_quotes(time_frame, year=2024, month=1, day=1, asset='EURUSD'):
#     try:
#         timezone = pytz.timezone('Europe/Paris')
#         time_from = datetime.datetime(year, month, day, tzinfo=timezone)
#         time_to = datetime.datetime.now(timezone)
        
#         symbol = assets[asset]
        
#         df = yf.download(symbol, start=time_from.strftime(
#             '%Y-%m-%d'), end=time_to.strftime('%Y-%m-%d'), interval=time_frame, progress=False )
        
#         if df.empty:
#             print(f'No data for {asset}. Check ticker symbol')
#             return pd.DataFrame()
        
#         df = df.rename(columns={'Open': 'open',
#                                 'High': 'high',
#                                 'Low': 'low',
#                                 'Close': 'close',
#                                 'Volume': 'tick_volume'})
#         df.reset_index(inplace=True)
#         return df
#     except Exception as e:
#         print(f"Error fetching data for {asset}: {e}")
#         return pd.DataFrame()
    
    
def get_quotes(time_frame, year=2005, month=1, day=1, asset='EURUSD'):
    try:
        timezone = pytz.timezone('Europe/Paris')
        time_from = datetime.datetime(year, month, day, tzinfo=timezone)
        time_to = datetime.datetime.now(timezone)

        symbol = assets[asset]
        df = yf.download(
            symbol,
            start=time_from.strftime('%Y-%m-%d'),
            end=time_to.strftime('%Y-%m-%d'),
            interval=time_frame,
            progress=False
        )

        if df.empty:
            print(f"No data for {asset}. Check ticker symbol or timeframe.")
            return pd.DataFrame()

        df = df.rename(columns={
            'Open': 'open',
            'High': 'high',
            'Low': 'low',
            'Close': 'close',
            'Volume': 'tick_volume'
        })

        df.reset_index(inplace=True)
        return df

    except Exception as e:
        print(f"Error fetching data for {asset}: {e}")
        return pd.DataFrame()


def mass_import(asset_index, time_frame):
    
    asset_list = list(assets.keys())
    asset = asset_list[asset_index]
    if time_frame == 'H1':
        data = get_quotes(frame_H1, 2013, 1, 1, asset=asset)
    elif time_frame == 'D1':
        data = get_quotes(frame_D1, 2000, 1, 1, asset=asset)
    else:
        data = get_quotes(frame_D1, 2000, 1, 1, asset=asset)

    if data.empty:
        print(f"No data found for {asset} on timeframe {time_frame}")
        return np.array([])

    # Return only OHLC like MT5 version
    data = data.loc[:, ['open', 'high', 'low', 'close']].values
    data = np.round(data, decimals=5)
    return data

In [15]:
my_data = mass_import(0, 'D1')

/var/folders/pp/x__1tf4n7j98nw_8bwlmgxj00000gn/T/ipykernel_26611/1567807126.py:35: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(


In [16]:
my_data

array([[1.2034 , 1.20401, 1.1944 , 1.1965 ],
       [1.1961 , 1.2109 , 1.1946 , 1.2089 ],
       [1.209  , 1.213  , 1.2077 , 1.2123 ],
       ...,
       [1.1618 , 1.16477, 1.16017, 1.1618 ],
       [1.16317, 1.16496, 1.16187, 1.16317],
       [1.16518, 1.16686, 1.16279, 1.16518]], shape=(5685, 4))

In [19]:
# Save to Excel (optional)
if my_data.size > 0:
    df = pd.DataFrame(my_data, columns=['Open', 'High', 'Low', 'Close'])
    df.to_excel('my_data.xlsx', index=False)
 
    
else:
    print("No data to save.")